In [ ]:
"""
@author Michael Levet
@author Jeremy Alm
@date 08/18/2022

This code tests whether the abstract relation algebra 
\mathcal{A}_{4}([i,i,i]) admits a finite representation 
on n points using a SAT solver. We define a Boolean formula 
\varphi whose satisfiability is a necessary condition to be 
representable on n points. So if \varphi is not satisfiable, 
then \mathcal{A}_{4}([i,i,i]) is not representable on n points.
"""

import boolexpr as bx
import itertools

import cProfile
import signal
from contextlib import contextmanager

In [ ]:
# change the value of n to test if 
# \mathcal{A}_{4}([i,i,i]) is representable on n points
n = 21

vertices = list(range(n))
edges = list(itertools.combinations(vertices, 2))
ctx = bx.Context()
variables = { **{ (i,j,0) : ctx.get_var(str((i,j,0))) for i,j in edges} ,
             **{ (i,j,1) : ctx.get_var(str((i,j,1))) for i,j in edges} ,  
             **{ (i,j,2) : ctx.get_var(str((i,j,2))) for i,j in edges},
            **{ (i,j,3) : ctx.get_var(str((i,j,3))) for i,j in edges}}


In [ ]:
# exactly one color is true
every_edge_colored = ( ((variables[(i,j,0)] | variables[(i,j,1)] | variables[(i,j,2)] | variables[(i,j,3)] )) 
                      for i,j in edges
                     )

conjuncts_colors = ( ~(variables[(i,j,0)] & variables[(i,j,1)]) &
                     ~(variables[(i,j,0)] & variables[(i,j,2)]) &
                     ~(variables[(i,j,0)] & variables[(i,j,3)]) & 
                     ~(variables[(i,j,1)] & variables[(i,j,2)]) &
                     ~(variables[(i,j,1)] & variables[(i,j,3)]) &
                     ~(variables[(i,j,2)] & variables[(i,j,3)])
                    for i,j in edges )

phi_0 = bx.and_s(bx.and_s(*every_edge_colored), bx.and_s(*conjuncts_colors))
print("Made phi_0")


# start with the atom a_0, which we call red. Then examine the needs of this red edge
R = {(0,1), (0, 2), (0, 3), (0, 4), (1,5), (1,6), (1,7)} # red edges/a_0
B = {(1, 2), (0, 5), (0,8),(1,8), (0,9), (0,10), (1,11), (1,12)} # blue edges/a_1
G = {(1,3), (0, 6), (1,9), (0, 11), (0, 13), (1,13), (0, 14), (1, 15)} # green edges/a_2
P = {(1,4), (0, 7), (1, 10), (0, 12), (1, 14), (0, 15), (0, 16), (1,16)} # purple edges/a_3

conjuncts_red_subgraph = (variables[(i,j,0)] for i,j in R)
conjuncts_blue_subgraph = (variables[(i,j,1)] for i,j in B)
conjuncts_green_subgraph = (variables[(i,j,2)] for i,j in G)
conjuncts_purple_subgraph = (variables[(i,j,3)] for i,j in P)

#color the existing edges by what we know
phi_1 = bx.and_s( bx.and_s(*conjuncts_red_subgraph), bx.and_s(*conjuncts_blue_subgraph),
                bx.and_s(*conjuncts_green_subgraph), bx.and_s(*conjuncts_purple_subgraph))
print("Made phi_1")

def disjunction(i,j,c1,c2):
    # returns a disjunction that says edge ij has need c1-c2 met
    return bx.or_s( *( variables[(min(i,k), max(i,k), c1)] 
                               & variables[(min(j,k), max(j,k), c2)] for k in set(range(n)) - {i,j} ) ) 

# each edge must have its "needs" met

#all possible needs for red edges
edge_needs_r = {(0,1),(1,0),(0,2),(2,0),(0,3),(1,3), (1,1),(2,2),(3,3), (1,2),(2,1),(1,3),(3,1),(1,3),(2,3)}

# all possible needs for blue edges
edge_needs_bl = {(1,0),(0,1),(2,1),(1,2),(0,3),(1,3), (0,0),(2,2),(3,3), (2,0),(0,2),(0,3),(3,0),(3,2),(2,3)}

# all possible needs for green edges
edge_needs_gr = {(2,0),(0,2),(1,2),(2,1),(1,3),(2,3), (0,0),(1,1),(3,3), (0,1),(1,0),(0,3),(3,0),(3,1),(3,2)}

#all possible needs for purple edges
edge_needs_p = {(1,0),(0,1),(2,0),(0,2),(0,3),(3,0), (0,0),(1,1),(2,2), (1,2),(2,1),(1,3),(3,1),(2,3),(3,2)}

print("Made phi_2")

#forbid monochromatic triangles
conjuncts_forbidden_triangles = ( ~(variables[(i,j,0)] & variables[(i,k,0)] & variables[(j,k,0)]) & 
                                  ~(variables[(i,j,1)] & variables[(i,k,1)] & variables[(j,k,1)]) &
                                  ~(variables[(i,j,2)] & variables[(i,k,2)] & variables[(j,k,2)]) &
                                  ~(variables[(i,j,3)] & variables[(i,k,3)] & variables[(j,k,3)])
                                 for i,j,k in itertools.combinations(vertices,3))

phi_3 = bx.and_s(*conjuncts_forbidden_triangles)
print("Made phi_3")


conjuncts_needs_red = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_r ))  
                       for i,j in R)

#every red edge must have its "needs" met
phi_2 = bx.and_s(*conjuncts_needs_red)


conjuncts_needs_Bl = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_bl ))  
                       for i,j in B)


#every blue edge must have its "needs" met
phi_4 = bx.and_s(*conjuncts_needs_Bl)
print("Made phi_4")

conjuncts_needs_Green = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_gr ))  
                       for i,j in G)


#every green edge must have its "needs" met
phi_5 = bx.and_s(*conjuncts_needs_Green)
print("Made phi_5")


conjuncts_needs_Purple = ( bx.and_s(*( disjunction(i,j,c1,c2) 
                              for c1,c2 in edge_needs_p ))  
                       for i,j in P)


#every green edge must have its "needs" met
phi_6 = bx.and_s(*conjuncts_needs_Purple)
print("Made phi_5")



# every edge not colored must have its "needs" met
edge_unknown = set(set(edges) - set(R) - set(B) - set(G))

def needs_r(i,j):
    # returns a disjunction that says edge ij is red and has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_r ))  

def needs_b(i,j):
    # returns a disjunction that says edge ij is blue has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_bl )) 

def needs_g(i,j):
    # returns a disjunction that says edge ij is blue has need c1-c2 met
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_gr )) 

def needs_p(i,j):
    return bx.and_s(*( disjunction(i,j,c1,c2) 
                      for c1,c2 in edge_needs_p )) 

conjuncts_needs_unknown =  (  (variables[(i,j,0)] & needs_r(i,j) ) |  
                                (variables[(i,j,1)] & needs_b(i,j) ) |  
                                (variables[(i,j,2)] & needs_g(i,j) ) |
                                (variables[(i,j,3)] & needs_p(i,j))
                              for i,j in edge_unknown)

phi_7 = bx.and_s(*conjuncts_needs_unknown)
print("Made phi_7 \n")


phi = bx.and_s(phi_0, phi_1, phi_2, phi_3, phi_4, phi_5, phi_6, phi_7)

print("Starting to simplify")
phi_simpl = phi.simplify()
print("Simplified")
phi_tseytin = phi.tseytin(ctx)
print("Tseytin Call Completed")
print("Call SAT")

phi_tseytin.sat()